# Macrophages (zebrafish)

In [ ]:
#%matplotlib nbagg
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
from os import listdir
from os.path import isfile, join
import re
import anndata
import seaborn as sns

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list(name='gene_cmap', colors=['lightgrey', 'thistle', 'red', 'darkred']) 

sc.settings.set_figure_params(dpi=80, color_map='viridis', vector_friendly=False,  dpi_save=300)
from matplotlib import font_manager
font_manager.fontManager.addfont('/cluster/groups/Wangxiaonan/share/yingjun/arial.ttf')
plt.rcParams['font.sans-serif'] = 'Arial'

# Recalculate UMAP

In [ ]:
adata = sc.read('./Final_write/ZF_mac.h5ad')

In [ ]:
adata.obs['Genotype'].value_counts()

In [ ]:
adata = anndata.AnnData(X=adata.raw.X.A, obs=adata.obs, var=adata.raw.var)
adata.raw = adata

In [ ]:
np.min(adata.raw.X)

In [ ]:
sc.pp.filter_genes(adata, min_cells=1)
sc.pp.highly_variable_genes(
    adata, min_mean=0.02, max_mean=3, min_disp=0.3, batch_key='StudyID', inplace=True)
sc.pl.highly_variable_genes(adata)
print(np.sum(adata.var.highly_variable))
adata = adata[:, adata.var.highly_variable].copy()
# scale for pca
sc.pp.scale(adata)
# pca
sc.tl.pca(adata, svd_solver='arpack')
sc.external.pp.harmony_integrate(adata, key='StudyID', adjusted_basis = 'X_pca_harmony_StudyID_combined')
sc.pp.neighbors(adata, use_rep = 'X_pca_harmony_StudyID_combined')
sc.tl.umap(adata)

In [ ]:
adata.write('./Final_write/ZF_mac.h5ad')

Matching macrophage cell-types between adult mice and zebrafish

# Scoring

In [ ]:
adata = sc.read('./Final_write/ZF_mac.h5ad')

In [ ]:
adata = anndata.AnnData(X=adata.raw.X,var=adata.raw.var,obs=adata.obs,obsm=adata.obsm)

In [ ]:
sc.pp.scale(adata,max_value=10)#Must scale!!!!

In [ ]:
np.max(adata.X)

In [ ]:
adata.shape

In [ ]:
gdict = pd.read_csv('./Final_Files/TransedMouse100Marker_Outer.csv',index_col=0)

In [ ]:
gdict.columns = ['Ma-Cycling', 'Ma-Endo', 'Ma-Fibro', 'Ma-Fn1', 'Ma-MD-Cxcl3',
       'Ma-MD-Isg15', 'Ma-MD-Mpeg1', 'Ma-MD-Prdx1', 'Ma-TR-Cd83+',
       'Ma-TR-Cd83-', 'Ma-TR-Cd163', 'Ma-lowGC', 'Ma-MD-Ccl2']

In [ ]:
gdict = gdict.to_dict()

In [ ]:
for k, v in gdict.items():
    sc.tl.score_genes(adata, gene_list=v.values(), score_name='Score_'+k, use_raw=False)

In [ ]:
sname = [x for x in adata.obs_keys() if 'Score_' in x]

In [ ]:
adata.obs['t2'] = np.std(adata.obs[sname], axis=1)

In [ ]:
s2 = adata.obs[sname].max(axis=1)

In [ ]:
adata.obs['s2'] = s2

In [ ]:
sc.pl.umap(adata,color='s2',cmap='bwr',title='Score',
          #save='_s2.pdf',
          )

In [ ]:
adata.obs['Score']=adata.obs.s2

In [ ]:
adata.obs.keys()

In [ ]:
ax = pd.DataFrame(adata.obs['Score']).plot.kde(xlim=(-0.5,1.5))
ax.axvline(0.05, linestyle='--', color='orange')
#plt.savefig('s2_density.pdf',bbox_inches='tight')

In [ ]:
Tn = adata.obs[sname].idxmax(axis=1)

In [ ]:
adata.obs['Tn'] = [re.sub('Score_', '',x) for x in Tn]

In [ ]:
# Filter out unidentified cells
tk = [adata.obs.Tn[x] if adata.obs.s2[x]>0.05 else 'NA' for x in adata.obs_names]

In [ ]:
adata.obs['Tn2'] = tk

In [ ]:
sc.pl.umap(adata,color=['Tn2'],size=3,title='Zebrafish CellType',
          #save='_fishCT3scored.pdf'
          )

In [ ]:
adata2 = adata[adata.obs.Tn2!='NA']

In [ ]:
cat = ['Ma-TR-Cd163', 'Ma-TR-Cd83+', 'Ma-TR-Cd83-', 'Ma-MD-Mpeg1',
       'Ma-MD-Prdx1', 'Ma-MD-Cxcl3', 'Ma-MD-Isg15', 'Ma-MD-Ccl2', 'Ma-Fn1', 'Ma-Fibro',
       'Ma-Endo', 'Ma-Cycling', 'Ma-lowGC']
adata2.obs['Tn2'] = adata2.obs.Tn2.astype('category').cat.reorder_categories(cat)

In [ ]:
adata2.write('./Final_write/ZF_mac_processed.h5ad')

In [ ]:
adata.write('./Final_write/ZF_mac_scored.h5ad')